In [26]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

In [5]:
try:
    movies_data = pd.read_csv('movies.csv')
    print("Dataset loaded successfully!")
except FileNotFoundError:
    print("Error: 'movies.csv' not found. Please make sure it's in the same folder as your script.")

Dataset loaded successfully!


In [10]:
print("First 5 rows of the dataset:")
movies_data.head()

First 5 rows of the dataset:


,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [11]:
print("\nShape of the dataset:", movies_data.shape)


Shape of the dataset: (4803, 24)


In [16]:
display (movies_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4803 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4803 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

None

In [15]:
#Preprocessing
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')
    
print("\nMissing values in selected features have been filled.")

['genres', 'keywords', 'tagline', 'cast', 'director']

Missing values in selected features have been filled.


In [17]:
display (movies_data[selected_features].head())

,genres,keywords,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [18]:
# --- Combine Features and Vectorize Text ---

print("\nCombining features into a single text string for each movie...")
movies_data['combined_features'] = movies_data['genres'] + ' ' + movies_data['keywords'] + ' ' + movies_data['tagline'] + ' ' + movies_data['cast'] + ' ' + movies_data['director']

print("First 5 combined features:")
print(movies_data['combined_features'].head())



Combining features into a single text string for each movie...
First 5 combined features:
0    Action Adventure Fantasy Science Fiction cultu...
1    Adventure Fantasy Action ocean drug abuse exot...
2    Action Adventure Crime spy based on novel secr...
3    Action Crime Drama Thriller dc comics crime fi...
4    Action Adventure Science Fiction based on nove...
Name: combined_features, dtype: object


In [20]:
vectorizer = TfidfVectorizer()

# Convert the combined text features into numerical feature vectors.
print("\nConverting text to numerical vectors using TF-IDF...")
feature_vectors = vectorizer.fit_transform(movies_data['combined_features'])

print("Conversion complete.")
print("Shape of feature vectors:", feature_vectors.shape)


Converting text to numerical vectors using TF-IDF...
Conversion complete.
Shape of feature vectors: (4803, 17318)


In [22]:
print("\nCalculating cosine similarity scores...")
similarity = cosine_similarity(feature_vectors)
print("Similarity calculation complete.")


Calculating cosine similarity scores...
Similarity calculation complete.


In [28]:
# --- Get movie recommendations ---

# Get a movie name from the user.
movie_name = input('\nEnter your favourite movie name: ')

# Find the closest match for the movie title given by the user.
list_of_all_titles = movies_data['title'].tolist()
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

if not find_close_match:
    print(f"Sorry, the movie '{movie_name}' was not found in our database.")
else:
    close_match = find_close_match[0]
    
    # Find the index of the movie with the title.
    index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
    
    # Get a list of similarity scores for that particular movie.
    # The enumerate function adds a counter (the movie index) to the list of scores.
    similarity_score = list(enumerate(similarity[index_of_the_movie]))
    
    # Sort the movies based on their similarity score in descending order.
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    
    # Print the top 20 similar movies.
    print(f'\nMovies suggested for you based on "{close_match}": \n')
    i = 1
    for movie in sorted_similar_movies:
        index = movie[0]
        title_from_index = movies_data[movies_data.index == index]['title'].values[0]
        
        # We don't want to recommend the movie itself, so we check if the index is different.
        if index != index_of_the_movie:
            print(f"{i}. {title_from_index}")
            i += 1
        
        if i > 20:
            break


Movies suggested for you based on "Iron Man": 

1. Iron Man 2
2. Iron Man 3
3. Avengers: Age of Ultron
4. The Avengers
5. Captain America: Civil War
6. Captain America: The Winter Soldier
7. Ant-Man
8. X-Men
9. Made
10. X-Men: Apocalypse
11. X2
12. The Incredible Hulk
13. The Helix... Loaded
14. X-Men: First Class
15. X-Men: Days of Future Past
16. Captain America: The First Avenger
17. Kick-Ass 2
18. Guardians of the Galaxy
19. Deadpool
20. Thor: The Dark World
